# Time to Trial End
Determine the value of hyperparameter `cnfg.TIME_TO_TRIAL_END_THRESHOLD`, which is used to decide if a fixation / visit is a _LWS_ instance or not.

In [82]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from statsmodels.sandbox.stats.stats_dhuard import percentileofscore

import config as cnfg

# pio.renderers.default = "notebook"
pio.renderers.default = "browser"

### Read data

In [2]:
from pipeline.read_data import read_saved_data
_targets, actions, _metadata, idents, fixations, visits = read_saved_data(cnfg.OUTPUT_PATH)

### Time From Last Action to Trial End

In [54]:
percentiles = [0.01, 0.05, 0.1, 0.25, 0.5]

last_action_times = actions.groupby(["subject", "trial"])["to_trial_end"].min()
action_summary = (
    pd.concat([
        last_action_times.describe(percentiles).rename("all"),
        last_action_times.groupby("subject").describe(percentiles).T,
    ], axis=1)
).T

print("Last Actions:")
print(f"99% of the **last actions** occur more than {action_summary.loc['all', '1%']:.2f} ms before trial end.")
action_summary

Last Actions:
99% of the **last actions** occur more than 561.39 ms before trial end.


,count,mean,std,min,1%,5%,10%,25%,50%,max
all,678.0,8179.536873,5227.353424,320.0,561.39,1027.85,1489.3,3692.00,7710.0,19657.0
2,59.0,9585.118644,4407.754984,1363.0,1407.66,2272.30,3676.0,6082.50,9556.0,17950.0
12,59.0,10111.457627,5344.531586,901.0,1070.36,1485.70,1920.6,6837.00,10016.0,18846.0
13,57.0,5727.491228,2666.537844,545.0,994.12,1484.20,2939.4,3724.00,5163.0,11747.0
14,57.0,9743.719298,4802.724773,692.0,754.72,1371.80,2717.8,6399.00,10717.0,19335.0
15,54.0,4801.740741,5795.877729,355.0,622.65,936.00,1027.3,1168.00,1756.0,19219.0
16,56.0,10205.196429,5929.390493,910.0,1214.70,1593.25,2435.5,4753.75,11040.5,19657.0
17,54.0,5592.537037,4569.873256,445.0,482.63,608.00,880.8,2180.00,3928.0,18134.0
18,60.0,7856.133333,4612.919512,320.0,454.52,890.40,2047.5,4279.00,7448.5,17895.0
19,59.0,6358.033898,5028.109705,563.0,572.86,739.00,868.0,2075.50,6355.0,18705.0


In [41]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        x=last_action_times,
        # y0="to_trial_end",
        name="All Subjects", legendgroup="All Subjects",
        text=actions.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Time to Trial End: {row['to_trial_end']:.2f} ms",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in actions[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = (
        last_action_times
        .xs(subj_id, level=cnfg.SUBJECT_STR)
        .reset_index(drop=False).assign(
            text=lambda df: df.apply(
                lambda row: f"{subj_string}<br>"
                            f"Trial: {row['trial']}<br>"
                            # f"Target: {row['target']}<br>"
                            f"Time to Trial End: {row['to_trial_end']:.2f} ms",
                axis=1
            ))
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="to_trial_end", x=subj_data["to_trial_end"],
            text=subj_data["text"],
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Last Action to Trial End", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()

### Identification-Fixation to Trial End

In [5]:
fixs_with_ident_time = fixations.copy()
dva_cols = [col for col in fixations.columns if col.endswith("distance_dva")]
fixs_with_ident_time["target"] = fixs_with_ident_time[dva_cols].idxmin(axis=1).str.replace("_distance_dva", "")
fixs_with_ident_time = (
    fixs_with_ident_time
    .drop(columns=[col for col in fixs_with_ident_time.columns if "_distance_" in col])
    .merge(
        idents.loc[
            idents["identification_category"] == "hit", ["subject", "trial", "target", "time"]
        ], on=["subject", "trial", "target"], how="left"
    )
)

fixs_with_ident_time.loc[:, "is_during"] = (fixs_with_ident_time["start_time"] <= fixs_with_ident_time["time"]) & (fixs_with_ident_time["time"] <= fixs_with_ident_time["end_time"])

In [72]:
last_identification_fixation = (
    fixs_with_ident_time
    .loc[fixs_with_ident_time["is_during"]]
    .groupby(["subject", "trial", "eye"])["to_trial_end"]
    .min()
    .reset_index(drop=False)
)

fixs_summary = (
    pd.concat([
        last_identification_fixation["to_trial_end"].describe(percentiles).rename("all"),
        last_identification_fixation.groupby("subject")["to_trial_end"].describe(percentiles).T,
    ], axis=1)
).T

print("Identification Fixations:")
print(f"99% of the **last identification fixations** occur more than {fixs_summary.loc['all', '1%']:.2f} ms before trial end.")
fixs_summary

Identification Fixations:
99% of the **last identification fixations** occur more than 131.72 ms before trial end.


,count,mean,std,min,1%,5%,10%,25%,50%,max
all,1244.0,8494.809486,5408.375474,5.0,131.72,598.30,1361.5,3798.50,7945.5,26464.0
2,105.0,10361.819048,4757.895161,784.0,848.76,2180.20,4230.0,7291.00,10267.0,20224.0
12,92.0,11075.869565,5034.215358,651.0,664.65,2991.65,4206.3,7317.50,10355.5,19238.0
13,108.0,6142.546296,3087.415119,496.0,551.44,2283.40,2917.6,3631.50,5517.0,13835.0
14,110.0,10484.136364,4647.034466,252.0,282.06,2114.90,4325.5,7463.25,10850.0,19297.0
15,102.0,4943.029412,5985.228411,5.0,8.68,416.10,485.1,600.00,1691.5,18750.0
16,112.0,9930.705357,5834.176513,51.0,151.88,1461.35,1848.4,5255.75,11060.0,19457.0
17,98.0,6469.500000,5573.385224,35.0,35.97,319.70,638.0,2272.25,5057.0,26464.0
18,118.0,7676.593220,4630.094039,6.0,50.71,683.65,1528.0,4534.25,7497.5,17672.0
19,109.0,6870.926606,5764.111691,12.0,20.00,240.00,421.0,1573.00,6666.0,21792.0


In [73]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        x=last_identification_fixation["to_trial_end"],
        y0="to_trial_end",
        name="All Subjects", legendgroup="All Subjects",
        text=last_identification_fixation.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Time to Trial End: {row['to_trial_end']:.2f} ms",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in last_identification_fixation[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = last_identification_fixation[last_identification_fixation[cnfg.SUBJECT_STR] == subj_id]
    texts = subj_data.apply(
        lambda row: f"{subj_string}<br>"
                    f"Trial: {row['trial']}<br>"
                    # f"Target: {row['target']}<br>"
                    f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
        axis=1
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="to_trial_end",
            x=subj_data["to_trial_end"],
            text=texts,
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Last Identification-Fixation to Trial End", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()

### All Pre-Identification-Visit to Trial End

In [97]:
pre_ident_visit = visits.loc[visits["before_identification"], ["subject", "trial", "eye", "duration", "to_trial_end"]]
visit_summary = pd.concat([
    pre_ident_visit["to_trial_end"].describe(percentiles).rename("all"),
    pre_ident_visit.groupby("subject")["to_trial_end"].describe(percentiles).T,
], axis=1).T

mean_dur, sd_dur = pre_ident_visit.duration.mean(), pre_ident_visit.duration.std()
mean_based_threshold = mean_dur + 2 * sd_dur

print("Pre-Identification Visits:")
print(f"Mean (SD) Duration:\t{mean_dur:.2f} ms ({sd_dur:.1f} ms)")
print(f"99% of the **pre-identification visits** occur more than {visit_summary.loc['all', '1%']:.2f} ms before trial end.")

print(f"Only {percentileofscore(pre_ident_visit["to_trial_end"], cnfg.TIME_TO_TRIAL_END_THRESHOLD) :.2f}% of pre-identification visits occur within {cnfg.TIME_TO_TRIAL_END_THRESHOLD} ms of trial end.")

print(f"Only {percentileofscore(pre_ident_visit["to_trial_end"], mean_based_threshold) :.2f}% of pre-identification visits occur within {mean_based_threshold :.2f} ms of trial end.")

visit_summary

Pre-Identification Visits:
Mean (SD) Duration:	315.41 ms (331.7 ms)
99% of the **pre-identification visits** occur more than 5.00 ms before trial end.
Only 4.45% of pre-identification visits occur within 1000 ms of trial end.
Only 4.35% of pre-identification visits occur within 978.83 ms of trial end.


,count,mean,std,min,1%,5%,10%,25%,50%,max
all,1295.0,10914.477220,5777.379299,5.0,5.00,1265.70,2958.0,6410.00,10912.0,25104.0
2,127.0,10680.370079,5214.529478,5.0,33.60,1399.60,3691.0,7579.00,9961.0,20022.0
12,122.0,14388.885246,4975.647908,1844.0,1896.19,7321.10,8358.8,10867.75,15236.5,21303.0
13,119.0,7539.781513,4011.232479,5.0,29.30,359.40,3595.6,4714.50,6660.0,15344.0
14,65.0,10266.630769,6248.580778,5.0,5.00,6.00,1007.0,5105.00,10925.0,21427.0
15,109.0,10312.889908,5462.070901,6.0,599.72,2463.20,2944.8,5357.00,10402.0,20070.0
16,107.0,10929.775701,6684.278205,5.0,5.00,94.20,1533.0,4238.00,12325.0,21160.0
17,89.0,9377.876404,5214.953272,811.0,811.00,1274.40,2524.0,5542.00,8371.0,20958.0
18,118.0,11000.169492,6293.367414,19.0,183.56,1420.00,2079.7,6416.00,9794.0,22631.0
19,93.0,10474.795699,5992.062695,6.0,1801.84,2618.20,3185.8,5004.00,10410.0,25104.0
